# Grouping Edits for File Classifications

In the case that you might want to edit a field uniformly, we can write a script that groups the acquisition files by certain fields. In this example, we show how to edit data by grouping.

First, we query flywheel for the full project:

In [29]:
import flywheel
import pandas as pd
from pandas.io.json.normalize import nested_to_record
import re
# add the script to the path
import sys
import os
sys.path.append(os.path.abspath("/home/ttapera/bids-on-flywheel/flywheel_bids_tools"))
import query_bids
import upload_bids
from tqdm import tqdm
import math

In [3]:
fw = flywheel.Client()
result = query_bids.query_fw("Q7 DSI", fw)

Convert this to a dataframe:

In [7]:
view = fw.View(columns='subject')
subject_df = fw.read_view_dataframe(view, result.id)
sessions = []
view = fw.View(columns='acquisition')
pbar = tqdm(total=100)

for ind, row in tqdm(subject_df.iterrows(), total=subject_df.shape[0]):
    session = fw.read_view_dataframe(view, row["subject.id"])
    if(session.shape[0] > 0):
        sessions.append(session)



  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

 12%|█▎        | 2/16 [00:00<00:01, 13.71it/s]

 25%|██▌       | 4/16 [00:00<00:00, 14.87it/s]

 38%|███▊      | 6/16 [00:00<00:00, 15.75it/s]

 50%|█████     | 8/16 [00:00<00:00, 15.94it/s]

 62%|██████▎   | 10/16 [00:00<00:00, 16.17it/s]

 75%|███████▌  | 12/16 [00:00<00:00, 16.54it/s]

 88%|████████▊ | 14/16 [00:00<00:00, 16.70it/s]

100%|██████████| 16/16 [00:00<00:00, 16.87it/s]



In [8]:
acquisitions = pd.concat(sessions)

And next, extract the acquisition's BIDS data.

A slight modification we add to the BIDS extractor function is adding the file classification, Series name, and TR (what we assume will be useful grouping criteria)

In [12]:
acquisitions

,acquisition.id,acquisition.label,acquisition.timestamp,acquisition.timezone,project.id,project.label,session.id,session.label,subject.id,subject.label
0,5c8011f0df93e3002efeb11e,Localizer,2018-02-10 19:36:23,America/New_York,5c7d7616df93e30028fc5227,Q7 DSI,5c8011f0df93e3002efeb11c,2018-02-10 14:21:55,5c8011f0df93e3002dfd44b8,18.02.10-13:38:26-STD-1.3.12.2.1107.5.2.43.66044
1,5c8011f0df93e3002efeb11f,t1w_mprage_0.9mm,2018-02-10 19:44:45,America/New_York,5c7d7616df93e30028fc5227,Q7 DSI,5c8011f0df93e3002efeb11c,2018-02-10 14:21:55,5c8011f0df93e3002dfd44b8,18.02.10-13:38:26-STD-1.3.12.2.1107.5.2.43.66044
2,5c8011f0df93e3002efeb120,t2w_space_0.9mm,2018-02-10 19:50:24,America/New_York,5c7d7616df93e30028fc5227,Q7 DSI,5c8011f0df93e3002efeb11c,2018-02-10 14:21:55,5c8011f0df93e3002dfd44b8,18.02.10-13:38:26-STD-1.3.12.2.1107.5.2.43.66044
3,5c8011f0df93e30029fce04e,DSI_730dir_b5000_mb3,2018-02-10 19:52:59,America/New_York,5c7d7616df93e30028fc5227,Q7 DSI,5c8011f0df93e3002efeb11c,2018-02-10 14:21:55,5c8011f0df93e3002dfd44b8,18.02.10-13:38:26-STD-1.3.12.2.1107.5.2.43.66044
4,5c8011f0df93e3002bfcf5ee,restingBOLD_mb6_1200,2018-02-10 20:45:51,America/New_York,5c7d7616df93e30028fc5227,Q7 DSI,5c8011f0df93e3002efeb11c,2018-02-10 14:21:55,5c8011f0df93e3002dfd44b8,18.02.10-13:38:26-STD-1.3.12.2.1107.5.2.43.66044
0,5c8018f7df93e30028fce906,Localizer,2018-02-10 21:20:29,America/New_York,5c7d7616df93e30028fc5227,Q7 DSI,5c8018f2df93e3002dfd4cd9,2018-02-10 16:14:31,5c8018f1df93e30029fce523,18.02.10-16:13:38-STD-1.3.12.2.1107.5.2.43.66044
1,5c8018f7df93e3002efec8ec,t1w_mprage_0.9mm,2018-02-10 21:28:10,America/New_York,5c7d7616df93e30028fc5227,Q7 DSI,5c8018f2df93e3002dfd4cd9,2018-02-10 16:14:31,5c8018f1df93e30029fce523,18.02.10-16:13:38-STD-1.3.12.2.1107.5.2.43.66044
2,5c8018f7df93e3002cfdfe53,t2w_space_0.9mm,2018-02-10 21:33:45,America/New_York,5c7d7616df93e30028fc5227,Q7 DSI,5c8018f2df93e3002dfd4cd9,2018-02-10 16:14:31,5c8018f1df93e30029fce523,18.02.10-16:13:38-STD-1.3.12.2.1107.5.2.43.66044
3,5c8018f7df93e30026fca6bd,DSI_730dir_b5000_mb3,2018-02-10 21:36:10,America/New_York,5c7d7616df93e30028fc5227,Q7 DSI,5c8018f2df93e3002dfd4cd9,2018-02-10 16:14:31,5c8018f1df93e30029fce523,18.02.10-16:13:38-STD-1.3.12.2.1107.5.2.43.66044
4,5c8018f7df93e3002afd0fa3,restingBOLD_mb6_1200,2018-02-10 22:29:07,America/New_York,5c7d7616df93e30028fc5227,Q7 DSI,5c8018f2df93e3002dfd4cd9,2018-02-10 16:14:31,5c8018f1df93e30029fce523,18.02.10-16:13:38-STD-1.3.12.2.1107.5.2.43.66044


In [37]:
def unlist_item(ls):
    
    if type(ls) is list:
        ls.sort()
        return(', '.join(x for x in ls))
    else:
        return float('nan')

def process_acquisition(acq_id, client):
    '''
    Extract an acquisition

    This function extracts an acquisition object and collects the important
    file classification information. These data are processed and returned as
    a pandas dataframe that can then be manipulated

    '''

    # get the acquisition object
    acq = client.get(acq_id)

    # convert to dictionary, and flatten the dictionary to avoid nested dicts
    files = [x.to_dict() for x in acq.files]
    flat_files = [nested_to_record(my_dict, sep='_') for my_dict in files]

    # define desirable columns in regex
    cols = r'(classification)|(^type$)|(^modality$)|(BIDS)|(RepetitionTime)|(SequenceName)|(SeriesDescription)'

    # filter the dict keys for the columns names
    flat_files = [
        {k: v for k, v in my_dict.items() if re.search(cols, k)}
        for my_dict in flat_files
        ]

    # add acquisition ID for reference
    for x in flat_files:
        x.update({'acquisition.id': acq_id})

    # to data frame
    df = pd.DataFrame(flat_files)

    # lastly, only pull niftis and dicoms; also convert list to string
    if 'type' in df.columns:
        df = df[df.type.str.contains(r'(nifti)|dicom')].reset_index(drop=True)
    list_cols = (df.applymap(type) == list).all()
    df.loc[:, list_cols] = df.loc[:, list_cols].applymap(unlist_item)
    return df

In [38]:
acq_dfs = []

for index, row in tqdm(acquisitions.iterrows(), total=acquisitions.shape[0]):
    
    try:
        temp = process_acquisition(row["acquisition.id"], fw)
        acq_dfs.append(temp)
    except:
        continue



  0%|          | 0/78 [00:00<?, ?it/s]/data/joy/BBL/applications/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


  1%|▏         | 1/78 [00:00<00:19,  3.98it/s]

  4%|▍         | 3/78 [00:00<00:09,  7.88it/s]

  6%|▋         | 5/78 [00:00<00:07,  9.87it/s]

  9%|▉         | 7/78 [00:00<00:06, 10.99it/s]

 12%|█▏        | 9/78 [00:00<00:05, 11.71it/s]

 14%|█▍        | 11/78 [00:00<00:05, 11.94it/s]

 17%|█▋        | 13/78 [00:01<00:05, 12.22it/s]

 19%|█▉        | 15/78 [00:01<00:05, 12.54it/s]

 22%|██▏       | 17/78 [00:01<00:04, 12.82it/s]

 24%|██▍       | 19/78 [00:01<00:04, 12.91it/s]

 27%|██▋       | 21/78 [00:01<00:04, 13.06it/s]

 29%|██▉       | 23/78 [00:01<00:04, 13.23it/s]

 32%|███▏      | 25/78 [00:01<00:03, 13.42it/s]

 35%|███▍      | 27/78 [00:01<00:03, 13.54it/s]

 37%|███▋      | 29/78 [00:02<00:03, 13.71it/s]

 40%|███▉      | 31/78 [00:02<00:03, 13.83it/s]

In [39]:
bids_data=pd.concat(acq_dfs, sort=False)
bids_data.head()

,acquisition.id,classification_Intent,classification_Measurement,info_RepetitionTime,info_SequenceName,info_SeriesDescription,modality,type
0,5c8011f0df93e3002efeb11e,Localizer,T2,40.00,*fl2d1,Localizer,MR,dicom
1,5c8011f0df93e3002efeb11e,Localizer,T2,0.04,_fl2d1,Localizer,MR,nifti
2,5c8011f0df93e3002efeb11e,Localizer,T2,0.04,_fl2d1,Localizer,MR,nifti
3,5c8011f0df93e3002efeb11e,Localizer,T2,0.04,_fl2d1,Localizer,MR,nifti
0,5c8011f0df93e3002efeb11f,Structural,T1,2500.00,*tfl3d1_16ns,t1w_mprage_0.9mm,MR,dicom


Now let's assume we want to group by the following:

In [40]:
groups_list = ['classification_Intent', 'classification_Measurement']

In order to reference back to our original data frame, we create a group ID based on the groupings. These can be as granular as necessary and have as many different groups as you'd like.

In [42]:
bids_data2 = bids_data.copy()

# figured out how to pipe pandas like R's "%>%". goddamn finally

bids_data2['group_id'] = (bids_data
                          # groupby and keep the columns as columns
                          .groupby(groups_list, as_index=False)
                          # index the groups
                          .ngroup()
                          .add(1))

In [43]:
bids_data2.head()

,acquisition.id,classification_Intent,classification_Measurement,info_RepetitionTime,info_SequenceName,info_SeriesDescription,modality,type,group_id
0,5c8011f0df93e3002efeb11e,Localizer,T2,40.00,*fl2d1,Localizer,MR,dicom,2
1,5c8011f0df93e3002efeb11e,Localizer,T2,0.04,_fl2d1,Localizer,MR,nifti,2
2,5c8011f0df93e3002efeb11e,Localizer,T2,0.04,_fl2d1,Localizer,MR,nifti,2
3,5c8011f0df93e3002efeb11e,Localizer,T2,0.04,_fl2d1,Localizer,MR,nifti,2
0,5c8011f0df93e3002efeb11f,Structural,T1,2500.00,*tfl3d1_16ns,t1w_mprage_0.9mm,MR,dicom,3


So this is where we group the data, and select a random exemplar from each group:

In [44]:
grouped_data = bids_data2.groupby(groups_list, as_index=False).nth(1).reset_index(drop=True)

In [45]:
grouped_data

,acquisition.id,classification_Intent,classification_Measurement,info_RepetitionTime,info_SequenceName,info_SeriesDescription,modality,type,group_id
0,5c8011f0df93e3002efeb11e,Localizer,T2,0.04,_fl2d1,Localizer,MR,nifti,2
1,5c8011f0df93e3002efeb11f,Structural,T1,2.50,_tfl3d1_16ns,t1w_mprage_0.9mm,MR,nifti,3
2,5c8011f0df93e3002efeb120,Structural,T2,3.20,_spc_314ns,t2w_space_0.9mm,MR,nifti,4
3,5c8011f0df93e3002bfcf5ee,Functional,T2*,0.50,epfid2d1_64,restingBOLD_mb6_1200,MR,nifti,1
4,5c8018f7df93e30026fca6bd,NaN,NaN,4300.00,ep_b3735#487,DSI_730dir_b5000_mb3,MR,dicom,0


This is what you would download from a grouped query. Note that this isn't *strictly* a grouped dataframe. We have effectively emulated grouping by dropping duplicate rows by specific columns.

Now, we modify some data in a copy of the query:

In [50]:
grouped_data_modified = grouped_data.copy()
grouped_data_modified.loc[grouped_data_modified['classification_Measurement'].isnull(), 'classification_Measurement'] = "Diffusion"
grouped_data_modified.loc[2, 'info_SequenceName'] = "SomeT1wSequence"
grouped_data_modified

,acquisition.id,classification_Intent,classification_Measurement,info_RepetitionTime,info_SequenceName,info_SeriesDescription,modality,type,group_id
0,5c8011f0df93e3002efeb11e,Localizer,T2,0.04,_fl2d1,Localizer,MR,nifti,2
1,5c8011f0df93e3002efeb11f,Structural,T1,2.50,_tfl3d1_16ns,t1w_mprage_0.9mm,MR,nifti,3
2,5c8011f0df93e3002efeb120,Structural,T2,3.20,SomeT1wSequence,t2w_space_0.9mm,MR,nifti,4
3,5c8011f0df93e3002bfcf5ee,Functional,T2*,0.50,epfid2d1_64,restingBOLD_mb6_1200,MR,nifti,1
4,5c8018f7df93e30026fca6bd,NaN,Diffusion,4300.00,ep_b3735#487,DSI_730dir_b5000_mb3,MR,dicom,0


We use our function to index the cells that have changed between the source and modified:

In [51]:
diff = upload_bids.get_unequal_cells(grouped_data_modified, grouped_data)
diff

[[2, 4], [4, 2]]

Here, we loop through each of the changes and create a dictionary where the `key` is the group that the change needs to be applied to, and the value is a tuple of the `column:new_value` pair.

In [53]:
changes = {}

for x in diff:
    
    key = grouped_data_modified.loc[x[0], 'group_id']
    val = (grouped_data_modified.columns[x[1]], grouped_data_modified.iloc[x[0], x[1]])
    changes.update({key: val})

changes

{4: ('info_SequenceName', 'SomeT1wSequence'),
 0: ('classification_Measurement', 'Diffusion')}

Now, using these indices, we can apply the changes to the groups in the full dataset:

In [54]:
for group, change in changes.items():
    
    bids_data2.loc[bids_data2['group_id'] == group, change[0]] = change[1]

In [56]:
bids_data2.head(10)

,acquisition.id,classification_Intent,classification_Measurement,info_RepetitionTime,info_SequenceName,info_SeriesDescription,modality,type,group_id
0,5c8011f0df93e3002efeb11e,Localizer,T2,40.00,*fl2d1,Localizer,MR,dicom,2
1,5c8011f0df93e3002efeb11e,Localizer,T2,0.04,_fl2d1,Localizer,MR,nifti,2
2,5c8011f0df93e3002efeb11e,Localizer,T2,0.04,_fl2d1,Localizer,MR,nifti,2
3,5c8011f0df93e3002efeb11e,Localizer,T2,0.04,_fl2d1,Localizer,MR,nifti,2
0,5c8011f0df93e3002efeb11f,Structural,T1,2500.00,*tfl3d1_16ns,t1w_mprage_0.9mm,MR,dicom,3
1,5c8011f0df93e3002efeb11f,Structural,T1,2.50,_tfl3d1_16ns,t1w_mprage_0.9mm,MR,nifti,3
0,5c8011f0df93e3002efeb120,Structural,T2,3200.00,SomeT1wSequence,t2w_space_0.9mm,MR,dicom,4
1,5c8011f0df93e3002efeb120,Structural,T2,3.20,SomeT1wSequence,t2w_space_0.9mm,MR,nifti,4
0,5c8011f0df93e30029fce04e,NaN,Diffusion,4300.00,ep_b610#41,DSI_730dir_b5000_mb3,MR,dicom,0
0,5c8011f0df93e3002bfcf5ee,Functional,T2*,500.00,epfid2d1_64,restingBOLD_mb6_1200,MR,dicom,1


Then, we apply the rest of our uploading process (finding the changes, checking that each change is valid by our heuristic, and uploading).

This process is wrapped up in two new CL tools: `group-query` and `ungroup-query`.